**Setting up Transformer**

In [1]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00


**Importing Transformer Library**

In [3]:
from transformers import pipeline

**Sentiment Analysis with the pipeline function**

In [4]:
sentiment = pipeline("sentiment-analysis")
sentiment([
    "I've been waiting for HuggingFace course my whole life",
    "I hate this so much",
])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9861214756965637},
 {'label': 'NEGATIVE', 'score': 0.9995144605636597}]

**Breakdown of What happens in the pipeline Function**

**-------> Tokenizer ------------> Model -------------> PostProcessing**

**Tokenization**

In [5]:
from transformers import AutoTokenizer

# distilbert-base-uncased-finetuned-sst-2-english is the checkpoint used by default for sentiment analysis in the pipeline
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for HuggingFace course my whole life",
    "I hate this so much",
]

inputs = tokenizer(raw_inputs, padding = True, truncation = True, return_tensors="pt")
# [padding = True]: Sentences with number of words less than the maximum number of word expected by the model is padded with zero
# [truncation = True] : Sentences with number of words greater than the maximum number of word expected by the model is truncated
# [return_tensors = "pt"] : Informs the tokenizer to return pytorch tensor

# Output
# [input_ids] : Contains the ids of the sentences passed to the tokenizer
# [attension_mask] : It indicates where padding has been applied so that the model does not pay attention to it

In [6]:
inputs

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005, 17662, 12172,  2607,
          2026,  2878,  2166,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   102,     0,     0,     0,
             0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

**Model**

In [8]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# The AutoModel class loads a model without its pretrained head
model = AutoModel.from_pretrained(checkpoint)

outputs = model(**inputs)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# Representation of sentences passed
print(outputs.last_hidden_state.shape) #torch.Size([batch size, sequence length, hidden size])

torch.Size([2, 14, 768])


In [12]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# The AutoModelForSequenceClassification class loads a model suitable for a particular task
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs = model(**inputs)

In [14]:
# Model Logits
print(outputs.logits)

tensor([[-2.1236,  2.1398],
        [ 4.2141, -3.4158]], grad_fn=<AddmmBackward0>)


**PostProcessing**

In [15]:
# To get the associated probabilities we apply a SoftMax layer onnthe logits
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=1)
print(predictions)

tensor([[1.3878e-02, 9.8612e-01],
        [9.9951e-01, 4.8549e-04]], grad_fn=<SoftmaxBackward0>)


In [16]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}